In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('C:/Users/farou/Downloads/mlProj/Ads Recommendation Project (1).csv')
display(df.head(10))

,Ads,Age,Gender,Religion,Sport,Medical_Condition,Weight,Nationality,Hobby,Field_of_Interest,Salary,Location,Class_Level,Dependencies,Ad_Relevance,image_path
0,Tech Gadget,28,Male,Muslim,Football,NaN,75,Egyptian,Coding,CS,15000,Sheikh Zayed,Middle,Spouse,1,images/images\Tech Gadget\5.jpeg
1,Organic Food,45,Female,Christian,NaN,Diabetes,82,Egyptian,Cooking,Medical,22000,Maadi,Upper,Children,1,images/images\Organic Food\5.jpg
2,Gym Membership,22,Male,Muslim,Running,NaN,68,Egyptian,Running,Engineering,8000,Dokki,Lower,NaN,1,images/images\Gym Membership\10.jpg
3,Luxury Car,52,Male,Muslim,NaN,Hypertension,90,Egyptian,Traveling,Business,85000,Cairo,Upper,Spouse,0,images/images\Luxury Car\11.jpg
4,Medical Insurance,60,Female,Muslim,Walking,Arthritis,70,Egyptian,Gardening,Medical,12000,Giza,Lower,Grandma,1,images/images\Medical Insurance\5.jpg
5,Engineering Tools,33,Male,Coptic,NaN,NaN,80,Egyptian,Robotics,Engineering,30000,6th of October,Middle,Children,1,images/images\Engineering Tools\11.jpeg
6,Fashion Magazine,25,Female,Muslim,NaN,NaN,58,Egyptian,Dancing,Arts,10000,Maadi,Lower,Spouse,0,images/images\Fashion Magazine\11.jpg
7,Healthy Snacks,38,Female,Muslim,Yoga,NaN,65,Egyptian,Yoga,CS,18000,Heliopolis,Middle,Mother,1,images/images\Healthy Snacks\10.jpg
8,Car Insurance,42,Male,Muslim,Football,NaN,85,Egyptian,Photography,Business,25000,Giza,Middle,Father,0,images/images\Car Insurance\2.jpeg
9,Kids Toys,35,Female,Christian,NaN,NaN,70,Egyptian,Painting,Education,14000,Dokki,Lower,Children,1,images/images\Kids Toys\7.jpg


In [8]:
from sklearn.preprocessing import OrdinalEncoder

columns_to_encode = [
    'Ads',
    'Gender',
    'Religion',
    'Sport',
    'Medical Condition',
    'Nationality',
    'Hobby',
    'Field of Interest',
    'Location',
    'Class Level',
    'Dependencies'
]

# Filter out columns that do not exist in the DataFrame
existing_columns = [col for col in columns_to_encode if col in df.columns]

# Initialize OrdinalEncoder
# handle_unknown='use_encoded_value' and unknown_value=-1 will assign -1 to categories not seen during fit
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Fit and transform the selected columns
encoded_features = oe.fit_transform(df[existing_columns])

# Create a DataFrame from the encoded features
# OrdinalEncoder produces one column per original categorical column
encoded_df = pd.DataFrame(encoded_features, columns=existing_columns)

# Drop the original categorical columns from the DataFrame
df_temp = df.drop(columns=existing_columns)

# Concatenate the original DataFrame (without original categorical columns) and the encoded DataFrame
df_encoded = pd.concat([df_temp, encoded_df], axis=1)

display(df_encoded.head())

,Age,Medical_Condition,Weight,Field_of_Interest,Salary,Class_Level,Ad_Relevance,image_path,Ads,Gender,Religion,Sport,Nationality,Hobby,Location,Dependencies
0,28,NaN,75,CS,15000,Middle,1,images/images\Tech Gadget\5.jpeg,9.0,1.0,2.0,3.0,0.0,6.0,11.0,5.0
1,45,Diabetes,82,Medical,22000,Upper,1,images/images\Organic Food\5.jpg,8.0,0.0,0.0,NaN,0.0,7.0,7.0,0.0
2,22,NaN,68,Engineering,8000,Lower,1,images/images\Gym Membership\10.jpg,3.0,1.0,2.0,4.0,0.0,22.0,2.0,NaN
3,52,Hypertension,90,Business,85000,Upper,0,images/images\Luxury Car\11.jpg,6.0,1.0,2.0,NaN,0.0,28.0,1.0,5.0
4,60,Arthritis,70,Medical,12000,Lower,1,images/images\Medical Insurance\5.jpg,7.0,0.0,2.0,7.0,0.0,13.0,4.0,2.0


In [9]:
columns_to_remove = [
    'Medical_Condition',
    'Field_of_Interest',
    'Class_Level'
]

# Drop these columns from df_encoded
df_encoded = df_encoded.drop(columns=columns_to_remove, errors='ignore')

display(df_encoded.head())

,Age,Weight,Salary,Ad_Relevance,image_path,Ads,Gender,Religion,Sport,Nationality,Hobby,Location,Dependencies
0,28,75,15000,1,images/images\Tech Gadget\5.jpeg,9.0,1.0,2.0,3.0,0.0,6.0,11.0,5.0
1,45,82,22000,1,images/images\Organic Food\5.jpg,8.0,0.0,0.0,NaN,0.0,7.0,7.0,0.0
2,22,68,8000,1,images/images\Gym Membership\10.jpg,3.0,1.0,2.0,4.0,0.0,22.0,2.0,NaN
3,52,90,85000,0,images/images\Luxury Car\11.jpg,6.0,1.0,2.0,NaN,0.0,28.0,1.0,5.0
4,60,70,12000,1,images/images\Medical Insurance\5.jpg,7.0,0.0,2.0,7.0,0.0,13.0,4.0,2.0


In [5]:
import os
images_folder = 'images/images'

# Initialize image_path as empty
df['image_path'] = ''

# Set random seed for reproducibility (optional)
np.random.seed(42)

# For each category, assign images randomly WITHOUT replacement
for category in df['Ads'].unique():
    # Get all rows with this category
    category_mask = df['Ads'] == category
    
    # Get list of images in this category's folder
    category_folder = os.path.join(images_folder, category)
    images = [f for f in os.listdir(category_folder) 
              if f.endswith(('.jpg', '.png', '.jpeg', '.JPG', '.PNG'))]
    
    num_rows = category_mask.sum()
    num_images = len(images)
    
    print(f"{category}: {num_rows} rows, {num_images} images")
    
    if num_images < num_rows:
        print(f"  ⚠️ Warning: Not enough images ({num_images}) for rows ({num_rows})!")
        # Shuffle and repeat if needed
        assigned_images = list(images)
        while len(assigned_images) < num_rows:
            assigned_images.extend(images)
        np.random.shuffle(assigned_images)
        assigned_images = assigned_images[:num_rows]
    else:
        # Shuffle the images and take exactly num_rows (no duplicates)
        np.random.shuffle(images)
        assigned_images = images[:num_rows]
    
    # Create full paths
    full_paths = [os.path.join(category_folder, img) for img in assigned_images]
    
    # Assign to the correct rows
    df.loc[category_mask, 'image_path'] = full_paths

# Save the updated CSV
df.to_csv('Ads Recommendation Project (1).csv', index=False)

Tech Gadget: 14 rows, 14 images
Organic Food: 15 rows, 15 images
Gym Membership: 15 rows, 15 images
Luxury Car: 15 rows, 15 images
Medical Insurance: 15 rows, 15 images
Engineering Tools: 14 rows, 14 images
Fashion Magazine: 14 rows, 14 images
Healthy Snacks: 14 rows, 14 images
Car Insurance: 14 rows, 14 images
Kids Toys: 15 rows, 15 images


In [7]:
print(df.head(10))

                 Ads  Age  Gender   Religion     Sport Medical_Condition  \
0        Tech Gadget   28    Male     Muslim  Football               NaN   
1       Organic Food   45  Female  Christian       NaN          Diabetes   
2     Gym Membership   22    Male     Muslim   Running               NaN   
3         Luxury Car   52    Male     Muslim       NaN      Hypertension   
4  Medical Insurance   60  Female     Muslim   Walking         Arthritis   
5  Engineering Tools   33    Male     Coptic       NaN               NaN   
6   Fashion Magazine   25  Female     Muslim       NaN               NaN   
7     Healthy Snacks   38  Female     Muslim      Yoga               NaN   
8      Car Insurance   42    Male     Muslim  Football               NaN   
9          Kids Toys   35  Female  Christian       NaN               NaN   

   Weight Nationality        Hobby Field_of_Interest  Salary        Location  \
0      75    Egyptian       Coding                CS   15000    Sheikh Zayed   
1  